# 08: Procesamiento de Boletines

**Propósito:** Este *notebook* consolida todos los archivos `boletines.csv` (crudos, por período) de `data/01_raw/` en un único *dataset* maestro.

**Proceso:**
1.  **Carga y Consolidación:** Llama a `load_all_bulletin_files` para cargar y **deduplicar** todos los boletines por `boletin_id`.
2.  **Procesamiento LLM:** Inicializa un cliente de Ollama y llama a la función `extraer_materia_llm` (de `src/processing_utils.py`) sobre la columna `materias_str`.
3.  **Extracción de Features:** Desempaca la respuesta JSON del LLM (ej. `ambitos_detectados`).
4.  **Guardado:** Guarda el archivo maestro (`boletines_master_clean.parquet`) en `data/02_processed/`.

**Advertencia de Memoria:** Este *notebook* carga el **texto completo** de todos los proyectos de ley en la RAM. El archivo de salida será de **varios Gigabytes**.

**Dependencias:**
* `data/01_raw/[periodo]/boletines.csv` (Múltiples archivos)
* `src/processing_utils.py` (Debe contener `load_all_bulletin_files` y `extraer_materia_llm`)

**Salidas (Artifacts):**
* `data/02_processed/boletines_master_clean.parquet`

In [7]:
import pandas as pd
from pathlib import Path
import sys
import logging
from tqdm.notebook import tqdm
import json
import nltk
from nltk.corpus import stopwords
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from hdbscan import HDBSCAN

# --- Configurar Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Importar lógica personalizada de /src ---
sys.path.append('../') 
try:
    from src.processing_utils import (
        load_all_bulletin_files
    )
except ImportError as e:
    logging.error(f"ERROR: No se pudieron importar las funciones desde /src. {e}")
    raise

# Registrar 'tqdm' con pandas
tqdm.pandas()

In [8]:
# --- 1. Configuración de Rutas y Constantes ---
ROOT = Path.cwd().parent
DATA_DIR_RAW = ROOT / "data" / "01_raw"
DATA_DIR_PROCESSED = ROOT / "data" / "02_processed"

DATA_DIR_PROCESSED.mkdir(parents=True, exist_ok=True)

embedding_model = "paraphrase-multilingual-mpnet-base-v2"

# Archivo de salida
OUTPUT_FILE = DATA_DIR_PROCESSED / "boletines_master_clean.parquet"

logging.info(f"Archivo de Salida: {OUTPUT_FILE}")

2025-11-13 12:42:03,433 - INFO - Archivo de Salida: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\02_processed\boletines_master_clean.parquet


## 1. Carga, Consolidación y Deduplicación

Cargamos todos los `boletines.csv` y los unificamos en un solo DataFrame. La función `load_all_bulletin_files` se encarga de la deduplicación por `boletin_id`.

In [3]:
df_full = load_all_bulletin_files(DATA_DIR_RAW)

if not df_full.empty:
    display(df_full.head())
    print(f"Dimensiones del DataFrame consolidado y deduplicado: {df_full.shape}")
else:
    logging.error("No se cargaron datos. Deteniendo el notebook.")

2025-11-13 12:41:18,386 - INFO - Buscando archivos 'boletines.csv'...
2025-11-13 12:41:18,390 - INFO - Encontrados 6 archivos. Cargando...
2025-11-13 12:41:18,553 - INFO - DataFrame consolidado creado con 3528 filas.
2025-11-13 12:41:18,553 - INFO - Deduplicando por 'boletin_id'...
2025-11-13 12:41:18,557 - INFO - DataFrame deduplicado tiene 3286 boletines únicos.


,boletin_id,titulo,fecha_ingreso,iniciativa,camara_origen,etapa,leynro,link_mensaje_mocion,autores_json,materias_str,materias_json,boletin_id_consultado
0,2625-07,Moderniza la normativa reguladora de los arren...,15/11/2000,Moción,Senado,Tramitación terminada,Ley Nº 19.866,http://www.senado.cl/appsenado/index.php?mo=tr...,"[""R\u00edos Santander, Mario""]",PREDIOS URBANOS,"[""PREDIOS URBANOS""]",2625
1,3107-05,Crea una bonificación a la contratación de man...,17/10/2002,Mensaje,C.Diputados,Tramitación terminada,Ley Nº 19.853,http://www.senado.cl/appsenado/index.php?mo=tr...,[],BONIFICACION POR CONTRATACIÓN DE MANO DE OBRA,"[""BONIFICACION POR CONTRATACI\u00d3N DE MANO D...",3107
2,2727-11,Sobre los derechos y deberes de las personas e...,12/06/2001,Mensaje,C.Diputados,Archivado,NaN,http://www.senado.cl/appsenado/index.php?mo=tr...,[],SALIDA DE TROPAS,"[""SALIDA DE TROPAS""]",2727
3,3039-07,Reforma constitucional que establece la obliga...,03/09/2002,Mensaje,C.Diputados,Tramitación terminada,Ley Nº 19.876,http://www.senado.cl/appsenado/index.php?mo=tr...,[],EDUCACIÓN; REFORMA CONSTITUCIONAL,"[""EDUCACI\u00d3N"", ""REFORMA CONSTITUCIONAL""]",3039
4,3147-10,Aprueba el acuerdo por el que se establece una...,03/12/2002,Mensaje,C.Diputados,Tramitación terminada,D.S. Nº28,http://www.senado.cl/appsenado/index.php?mo=tr...,[],"ACUERDO DE ASOCIACIÓN CHILE-UNIÓN EUROPEA, SUS...","[""ACUERDO DE ASOCIACI\u00d3N CHILE-UNI\u00d3N ...",3147


Dimensiones del DataFrame consolidado y deduplicado: (3286, 12)


## 2. Procesamiento Bertopic: Clasificación de Materias

Aplicamos el modelo Bertopic a nuestras `materias_str`

In [4]:
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    logging.error(f"ERROR: No se pudieron encontrar las stopwords. Serán descargadas!")
    nltk.download('stopwords')

idioma = 'spanish'

try:
    logging.info("Definimos las palabras a ignorar.")
    logging.info(f"Idioma: {idioma}")
    spanish_stop_words = stopwords.words(idioma)

    logging.info("Definimos el contador de palabras.")
    vectorizer_model = CountVectorizer(stop_words=spanish_stop_words)

    logging.info("Definimos el reductor de dimensionalidad (UMAP).")
    umap_model = UMAP(n_neighbors=15, 
                      n_components=5, 
                      min_dist=0.0, 
                      metric='cosine')
    logging.info("Definimos el clusterizador (HDBSCAN).")
    hdbscan_model = HDBSCAN(min_cluster_size=25, 
                            metric='euclidean', 
                            cluster_selection_method='eom', 
                            prediction_data=True,
                            gen_min_span_tree=True)

    logging.info("Definimos el clasificador de tópicos.")
    topic_model = BERTopic(
        embedding_model=embedding_model,
        vectorizer_model=vectorizer_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        language="multilingual",
        verbose=True
    )
except Exception as e:
    logging.error(f"No se pudieron definir los parámetros {e}")

2025-11-13 12:41:18,582 - INFO - Definimos las palabras a ignorar.
2025-11-13 12:41:18,583 - INFO - Idioma: spanish
2025-11-13 12:41:18,587 - INFO - Definimos el contador de palabras.
2025-11-13 12:41:18,587 - INFO - Definimos el reductor de dimensionalidad (UMAP).
2025-11-13 12:41:18,588 - INFO - Definimos el clusterizador (HDBSCAN).
2025-11-13 12:41:18,589 - INFO - Definimos el clasificador de tópicos.


In [25]:
try:
    logging.info("Extrayendo tópicos!")
    docs = df_full['materias_str'].fillna('')
    topics, probs = topic_model.fit_transform(docs)
    topics_info = topic_model.get_topic_info()
    display(topics_info.head(4))
except Exception as e:
    logging.error(e)

2025-11-13 12:55:58,669 - INFO - Extrayendo tópicos!
2025-11-13 12:55:58,672 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/103 [00:00<?, ?it/s]

2025-11-13 12:56:08,235 - BERTopic - Embedding - Completed ✓
2025-11-13 12:56:08,236 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-13 12:56:24,496 - BERTopic - Dimensionality - Completed ✓
2025-11-13 12:56:24,497 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-13 12:56:24,607 - BERTopic - Cluster - Completed ✓
2025-11-13 12:56:24,611 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-13 12:56:24,676 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,Representative_Docs
0,-1,677,-1_sector_público_remuneraciones_salud,"[sector, público, remuneraciones, salud, reaju...",[AGUINALDO SECTOR PÚBLICO; FUNCIONARIOS PÚBLIC...
1,0,546,0_aguas_pesca_acuicultura_sanitarios,"[aguas, pesca, acuicultura, sanitarios, artesa...",[ACUICULTURA; PESCA Y ACUICULTURA; REGISTRO PE...
2,1,384,1____,"[, , , , , , , , , ]","[, , ]"
3,2,205,2_chile_acuerdo_santiago_bomberos,"[chile, acuerdo, santiago, bomberos, tratado, ...",[SEGUNDO PROTOCOLO COMPLEMENTARIO AL TRATADO D...


In [27]:
topics_map = topics_info.set_index('Topic')['Name'].to_dict()

In [29]:
df_full['topic_materia_id'] = topics
df_full['topic_materia_nombre'] = df_full['topic_materia_id'].apply(lambda x: topics_map[x])

## 3. Procesamiento Bertopic: Clasificación de títulos

Aplicamos nuestro modelo Bertopic a los títulos de los boletines

In [ ]:
try:
    logging.info("Extrayendo tópicos!")
    docs = df_full['titulo']
    topics, probs = topic_model.fit_transform(docs)
    topics_info = topic_model.get_topic_info()
    display(topics_info.head(4))
except Exception as e:
    logging.error(e)

2025-11-13 12:58:41,196 - INFO - Extrayendo tópicos!
2025-11-13 12:58:41,199 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/103 [00:00<?, ?it/s]

2025-11-13 12:58:55,952 - BERTopic - Embedding - Completed ✓
2025-11-13 12:58:55,952 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


In [ ]:
topics_map = topics_info.set_index('Topic')['Name'].to_dict()

In [ ]:
df_full['topic_titulo_id'] = topics
df_full['topic_titulo_nombre'] = df_full['topic_materia_id'].apply(lambda x: topics_map[x])

## 4. Guardamos los datos

In [ ]:
try:
    df_full.to_parquet(OUTPUT_FILE, index=False)
    logging.info(f"Guardado exitosamente: {OUTPUT_FILE}")
    logging.info(f"Dimensiones del DataFrame maestro: {df_full.shape}")
    
    print("\n--- Columnas Finales del DataFrame Limpio ---")
    print(df_full.columns.tolist())
    
except Exception as e:
    logging.error(f"ERROR al guardar en Parquet: {e}")

display(df_full.head())